In [2]:
%load_ext autoreload
%autoreload 2

# Imports

In [17]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [18]:
from dataloading import get_labeled_data

# Load the data

In [4]:
# file paths
train_path = "../data/preprocessed/train_vectors.csv"
scaled_train_path = "../data/preprocessed/train_vectors_scaled.csv"
valid_path = "../data/preprocessed/validation_vectors.csv"
scaled_valid_path = "../data/preprocessed/validation_vectors_scaled.csv"

In [5]:
# load the dataframes
train_df = pd.read_csv(train_path)
train_df_scaled = pd.read_csv(scaled_train_path)
valid_df = pd.read_csv(valid_path)
valid_df_scaled = pd.read_csv(scaled_valid_path)

In [6]:
# using the scaled dataframe
train_df_scaled.head()

,Port number,Packet speed,Data speed,Avg packet len,Source IP count,victim IP num,time_of_day,Significant flag,CLDAP,Generic UDP,...,Sentinel,IPv4 fragmentation,Suspicious traffic,SSDP,TCP Anomaly,SNMP,DNS,is_weekday,Type,is_synthetic
0,-0.817317,-0.258647,-0.114203,0.752471,-0.179160,-0.604654,0.597073,0,0,0,...,0,0,0,0,0,0,0,1,Normal traffic,False
1,-0.817317,-0.195674,0.024471,0.978049,-0.191742,-0.604474,0.670414,0,0,0,...,0,0,0,0,0,0,0,1,Normal traffic,False
2,-0.941431,-0.225965,-0.065259,0.781814,-0.191742,-0.604294,0.680810,0,0,0,...,0,0,0,0,0,0,0,1,Normal traffic,False
3,-0.941431,-0.178138,-0.008159,0.781814,-0.191742,-0.604294,0.680854,0,0,0,...,0,0,0,0,0,0,0,1,Normal traffic,False
4,-0.817317,-0.227559,-0.016316,0.941370,-0.191742,-0.604474,0.697654,0,0,0,...,0,0,0,0,0,0,0,1,Normal traffic,False


In [7]:
# get target features
data_df = train_df_scaled.drop(columns=['is_synthetic','Type'])
target_df = train_df_scaled['Type']

# encode target labels
target_df, target_label_encoder = get_labeled_data(pd.DataFrame(target_df))

X = data_df.to_numpy()
y = target_df.to_numpy()

In [24]:
# train - test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [28]:
# Initialize RandomForestClassifier
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)

# Train the classifier
rf_classifier.fit(X_train, y_train)

# Predict on the testing data
y_pred = rf_classifier.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

<ipython-input-28-a62efa131465>:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rf_classifier.fit(X_train, y_train)


Accuracy: 0.9627681296080426


In [35]:
X_valid = valid_df_scaled.drop(columns=['Type'])#.to_numpy()
len(data_df.columns)

26

In [36]:
data_df.columns

Index(['Port number', 'Packet speed', 'Data speed', 'Avg packet len',
       'Source IP count', 'victim IP num', 'time_of_day', 'Significant flag',
       'CLDAP', 'Generic UDP', 'CoAP', 'ACK Attack', 'NTP', 'SYN Attack',
       'RPC', 'CHARGEN', 'RDP', 'High volume traffic', 'Sentinel',
       'IPv4 fragmentation', 'Suspicious traffic', 'SSDP', 'TCP Anomaly',
       'SNMP', 'DNS', 'is_weekday'],
      dtype='object')

In [37]:
X_valid.columns

Index(['Port number', 'Significant flag', 'Packet speed', 'Data speed',
       'Avg packet len', 'Source IP count', 'Memcached', 'RPC', 'CHARGEN',
       'TFTP', 'Suspicious traffic', 'Sentinel', 'SYN Attack', 'CLDAP', 'WSD',
       'IPv4 fragmentation', 'High volume traffic', 'RIP', 'ICMP',
       'Generic UDP', 'SSDP', 'NTP', 'DNS', 'TCP Anomaly', 'SNMP', 'CoAP',
       'RDP', 'victim IP num', 'is_weekday', 'time_of_day'],
      dtype='object')

In [40]:
diff = []
for col in X_valid.columns:
    if col not in data_df.columns:
        diff.append(col)
diff

['Memcached', 'TFTP', 'WSD', 'RIP', 'ICMP']

In [41]:
diff2 = []
for col in data_df.columns:
    if col not in X_valid.columns:
        diff2.append(col)
diff2

['ACK Attack']

In [48]:
X_valid.drop(columns=diff, inplace=True)
X_valid['Ack Attack'] = 0

In [56]:
X_valid = X_valid.to_numpy()

In [59]:
y_valid = valid_df_scaled['Type']
target_label_encoder.transform(y_valid.values)

NotFittedError: This LabelEncoder instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.

In [55]:
y_pred = rf_classifier.predict(X_valid.to_numpy())

# Evaluate the model
accuracy = accuracy_score(valid_df_scaled['Type'].to_numpy(), y_pred)
print("Accuracy:", accuracy)

KeyboardInterrupt: 